# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [50]:
city_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,katsuura,35.1333,140.3000,298.84,88,98,6.84,JP,1624158960
1,1,rikitea,-23.1203,-134.9692,297.23,75,2,7.90,PF,1624158961
2,2,yar-sale,66.8333,70.8333,285.21,54,94,5.68,RU,1624158961
3,3,puerto ayora,-0.7393,-90.3518,296.23,90,87,2.68,EC,1624158962
4,4,awjilah,29.1081,21.2869,295.03,50,0,3.71,LY,1624158962


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
coord = city_data[['Lat','Lng']]
humid = city_data['Humidity']
heat_layer = gmaps.heatmap_layer(coord, weights = humid, dissipating = False, max_intensity = 100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:
#Define weather conditions for the reduced dataframe

#Humidity less than 30%

#Wind Speed less than 10 mph

#Cloudiness lower than 50 percent

In [63]:
vacation_city = city_data.loc[(city_data['Humidity'] < 30) &
                              (city_data['Wind Speed'] < 10) & 
                              (city_data['Cloudiness'] < 30)]
vacation_city = vacation_city.reset_index(drop = True)
vacation_city

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,45,saskylakh,71.9167,114.0833,303.29,21,20,7.03,RU,1624158983
1,84,chokurdakh,70.6333,147.9167,293.65,29,27,5.39,RU,1624159007
2,86,zabol,32.2500,67.2500,296.59,12,0,6.23,AF,1624159008
3,87,ojinaga,29.5667,-104.4167,308.68,15,1,3.60,MX,1624159008
4,94,komsomolskiy,40.4272,71.7189,299.83,25,4,2.06,UZ,1624159012
5,109,buraydah,26.3260,43.9750,297.40,12,0,0.40,SA,1624158999
6,113,riyadh,24.6877,46.7219,301.18,11,0,1.54,SA,1624159021
7,168,ignacio zaragoza,28.3833,-107.2167,294.48,20,22,4.37,MX,1624159049
8,169,birjand,32.8663,59.2211,298.41,16,0,2.06,IR,1624159050
9,211,dehloran,32.6941,47.2679,305.56,12,0,0.64,IR,1624159074


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [75]:
hotel_df = vacation_city
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,45,saskylakh,71.9167,114.0833,303.29,21,20,7.03,RU,1624158983,
1,84,chokurdakh,70.6333,147.9167,293.65,29,27,5.39,RU,1624159007,
2,86,zabol,32.2500,67.2500,296.59,12,0,6.23,AF,1624159008,
3,87,ojinaga,29.5667,-104.4167,308.68,15,1,3.60,MX,1624159008,
4,94,komsomolskiy,40.4272,71.7189,299.83,25,4,2.06,UZ,1624159012,


'71.9167,114.0833'

In [76]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius": 5000,
           "type": "lodging",
           "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    #get location from df
    params["location"] = f"{hotel_df['Lat'][index]},{hotel_df['Lng'][index]}"

    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"The first hotel within 5000m of {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"No hotel found in 5000 metres for {row['City']}")
        
    print("--------------------------------------------")


No hotel found in 5000 metres for saskylakh
--------------------------------------------
No hotel found in 5000 metres for chokurdakh
--------------------------------------------
No hotel found in 5000 metres for zabol
--------------------------------------------
The first hotel within 5000m of ojinaga is Suites Las Palmas.
--------------------------------------------
The first hotel within 5000m of komsomolskiy is Ikathouse.
--------------------------------------------
The first hotel within 5000m of buraydah is فندق راحة السلام.
--------------------------------------------
The first hotel within 5000m of riyadh is InterContinental Riyadh, an IHG Hotel.
--------------------------------------------
No hotel found in 5000 metres for ignacio zaragoza
--------------------------------------------
The first hotel within 5000m of birjand is Birjand Tourist Hotel.
--------------------------------------------
The first hotel within 5000m of dehloran is کافه گیم استورم.
------------------------

In [79]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [80]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(locations, info_box_content = hotel_info, fill_color = 'red')
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))